# Set Random Training Dataset

In [1]:
import os, sys, time, csv, random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
#printmd('**bold**')

In [3]:
data_path = "mlabel_corpora/JokeSkill.txt"
outp_path = "mlabel_corpora/JokeSkill_train_random.txt"

In [4]:
# set global variables: df
df = pd.read_csv(data_path, delimiter="\t")
#data_raw = df.loc[np.random.choice(data_raw.index, size=2000)]
print(df.shape) # same as data_raw.shape in Jupyter

(3365, 11)


In [5]:
from sklearn.model_selection import train_test_split

# ID=L1850 為分界，之前：吳玟萱，之後：黃亭筠，均為中文系同一屆
train, test = train_test_split(df, train_size=1691, shuffle=False) 
# (tempararily) set global variables: train, test 

print(train.shape)
print(test.shape)

(1691, 11)
(1674, 11)


/Users/sam/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


## Now randomly assign labels to the training set and save it to file

1. First, to know the original training label distribution.
2. Set the labels based on this distribution, by apply an algorithm from stackoverflow.com

In [6]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.copy.html
train_org = train.copy()

In [7]:
Total_Samples = len(train.index) # train.shape[0] # both work, but len(train.index) is faster
cat = list(train.columns.values)
print(Total_Samples, "\n", cat)

1691 
 ['ID', 'Title', 'Content', 'Pun', 'Exaggeration', 'Anthropomorphism', 'Bridge_Inference', 'Illogical', 'Irony', 'Imitation', 'Others']


In [8]:
matrix = train.iloc[:, 3:].values # get the values of the labels in matrix form
'''
matrix = np.array( # for testing
    [[0, 0, 1], 
     [1, 1, 0], 
     [1, 0, 0]]
)
'''
matrix

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [45]:
# Randomize matrix keeping row and column totals the same
# https://stackoverflow.com/questions/2133268/randomize-matrix-in-perl-keeping-row-and-column-totals-the-same

def shuffle(array):
    i = len(array)
    j = 0
    for _ in (array):
        i -= 1;
        j = random.randrange(0, i+1) #int rand($i + 1);
        #print('arrary:', array)
        #print(f'len(array)={len(array)}, (i, j)=({i}, {j})')
        if i != j: 
            tmp = array[i]
            array[i] = array[j]
            array[j] = tmp
    return array

def other_edits(matrix, cell, step, last_j):
    # We have succeeded if we've already made 3 edits.
    step += 1
    if step > 3: 
        return True

    # Determine the roster of next edits to fix the row or
    # column total upset by our prior edit.
    (i, j) = cell
    fixes = []
    if (step == 1):
        fixes = [[i, x] for x in range(len(matrix[0])) if x != j and not matrix[i][x] ]
        fixes = shuffle(fixes)
    elif (step == 2):
        fixes = [[x, j] for x in range(len(matrix)) if x != i and matrix[x][j]]
        fixes = shuffle(fixes)
    else:
        # On the last edit, the column of the fix must be
        # the same as the column of the initial edit.
        if not matrix[i][last_j]: fixes = [[i, last_j]]

    for f in (fixes):
        # If all subsequent fixes succeed, we are golden: make
        # the current fix and return true.
        if ( other_edits(matrix, f, step, last_j) ):
            matrix[f[0]][f[1]] = 0 if step == 2 else 1
            return True

    # Failure if we get here.
    return False # return False

def cells_to_move(matrix):
    # Returns a list of non-empty cells.
    i = -1
    cells = []
    for row in matrix:
        i += 1;
        for j in range(len(row)):
            if matrix[i][j]: cells.append([i, j])
    return cells

def edit_matrix(matrix):
    # Takes a matrix and moves all of the non-empty cells somewhere else.
    move_these = cells_to_move(matrix)
    for cell in move_these:
        (i, j) = cell
        # Move the cell, provided that the cell hasn't been moved
        # already and the subsequent edits don't lead to a dead end.
        if matrix[i][j] and other_edits(matrix, cell, 0, j):
            matrix[i][j] = 0
    return matrix

def init_matrix(rows, cols, density): # not used
    matrix = []
    for r in range(rows):
        matrix.append([ 1 if random.random() < density else 0  for _ in range(cols) ])
    return matrix

def Shuffle_Matrix(matrix, N, M, n_iter):
    #matrix = init_matrix(N, M, density)
    print("init_matrix:\n", matrix);
    for n in range(n_iter):
        print(f'iteration: {n+1}') # Show progress.
        matrix = edit_matrix(matrix)
        #print('matrix:\n', matrix)
    return matrix

def compute_density(train_org, cat):
    sum = 0
    for category in cat[3:]:
        sum += train_org[category].sum()
    density = sum/(train_org.shape[0] * train_org.shape[1])
    print(f'sum={sum}, density={density}')
    return density

print(matrix.shape[0], matrix.shape[1]) #;print(matrix)
#density = compute_density(train_org, cat)
#density = sum([sum(row) for row in matrix])/(matrix.shape[0] * matrix.shape[1])
#print('density : %1.6f'%density)

# Args: N rows, N columns, density, N iterations.
matrix2 = Shuffle_Matrix(matrix, matrix.shape[0], matrix.shape[1], 11) 
# even n_iter number would be the same for the toy 3x3 example
# n_iter =  1, both==380, 370
# n_iter =  5, both==349, 363, 371, 402, 390, 401, 371, 366, 381, 373, 389, 410, 384
# n_iter = 10, both==381, 392, 393, 364, 402, 399, 406, 386, 380
# n_iter = 11, both==350, 364, 362
# n_iter = 20, both==374, 356, 368
# n_iter = 21, both==396, 337, 379, 368
# n_iter = 30, both==384, 379
print("matrix2:\n", matrix2)

1691 8
init_matrix:
 [[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
matrix2:
 [[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


In [46]:
# set matrix2 to train
j = -1
for c in cat[3:]:
    j += 1
    for i in range(Total_Samples):
        train.at[i, c] = matrix2[i][j]

In [47]:
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(df2,
                              indicator=True,
                              how='outer')
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    #diff_df.to_csv('data/diff.csv')
    return diff_df

diff_both = dataframe_difference(train_org, train, which='both')
both=diff_both['_merge'].value_counts(); print(both)
diff = dataframe_difference(train_org, train)
one = diff['_merge'].value_counts(); print(one)
print(train.shape[0], both['both'], one['right_only'], one['left_only'])
print(both['both']+one['right_only'], both['both']/train.shape[0])

both          362
right_only      0
left_only       0
Name: _merge, dtype: int64
right_only    1329
left_only     1329
both             0
Name: _merge, dtype: int64
1691 362 1329 1329
1691 0.21407451212300413


### 1.1. Checking for missing values

In [48]:
missing_values_check = train_org.isnull().sum()
print(missing_values_check, "\n")
missing_values_check = train.isnull().sum()
print(missing_values_check)

ID                  0
Title               0
Content             0
Pun                 0
Exaggeration        0
Anthropomorphism    0
Bridge_Inference    0
Illogical           0
Irony               0
Imitation           0
Others              0
dtype: int64 

ID                  0
Title               0
Content             0
Pun                 0
Exaggeration        0
Anthropomorphism    0
Bridge_Inference    0
Illogical           0
Irony               0
Imitation           0
Others              0
dtype: int64


### 1.2. Calculating number of jokes under each label

In [49]:
# Jokes with no label are considered to be clean jokes.
# Creating seperate column in dataframe to identify clean jokes.
# We use axis=1 to count row-wise and axis=0 to count column wise
def print_empty_label(df, s):
    rowSums = df.iloc[:,3:].sum(axis=1)
    #print(rowSums.shape)
    #print(rowSums.head())
    clean_comments_count = (rowSums==0).sum(axis=0)

    print(f"Total number of {s} jokes = ",len(df))
    print(f"Number of clean jokes in {s} = ",clean_comments_count)
    print(f"Number of {s} jokes with labels =",(len(df)-clean_comments_count))
    print()

In [50]:
print(train_org.equals(train))
print_empty_label(train_org, 'train_org')
print_empty_label(train, 'train')

False
Total number of train_org jokes =  1691
Number of clean jokes in train_org =  2
Number of train_org jokes with labels = 1689

Total number of train jokes =  1691
Number of clean jokes in train =  2
Number of train jokes with labels = 1689



In [51]:
# set global variables: categories
categories = list(df.columns.values)
print(categories)
categories = categories[3:]
print(categories)

['ID', 'Title', 'Content', 'Pun', 'Exaggeration', 'Anthropomorphism', 'Bridge_Inference', 'Illogical', 'Irony', 'Imitation', 'Others']
['Pun', 'Exaggeration', 'Anthropomorphism', 'Bridge_Inference', 'Illogical', 'Irony', 'Imitation', 'Others']


In [52]:
# Calculating number of jokes in each category
def print_category_count(df, categories):
    counts = []
    for category in categories:
        counts.append((category, df[category].sum()))
    df_stats = pd.DataFrame(counts, columns=['category', 'number of jokes'])
    print(df_stats)
    print()

In [53]:
print_category_count(df, categories)
print_category_count(train_org, categories)
print_category_count(train, categories)
print_category_count(test, categories)

           category  number of jokes
0               Pun             1039
1      Exaggeration              131
2  Anthropomorphism              180
3  Bridge_Inference              607
4         Illogical              924
5             Irony               82
6         Imitation              258
7            Others              588

           category  number of jokes
0               Pun              778
1      Exaggeration               71
2  Anthropomorphism              140
3  Bridge_Inference              316
4         Illogical              438
5             Irony               47
6         Imitation              148
7            Others              110

           category  number of jokes
0               Pun              778
1      Exaggeration               71
2  Anthropomorphism              140
3  Bridge_Inference              316
4         Illogical              438
5             Irony               47
6         Imitation              148
7            Others              110

In [ ]:
def plot_category_count(df, categories):
    sns.set(font_scale = 2)
    plt.figure(figsize=(15,8))

    ax= sns.barplot(categories, df.iloc[:,3:].sum().values)

    plt.title("Jokes in each category", fontsize=24)
    plt.ylabel('Number of jokes', fontsize=18)
    plt.xlabel('Joke Skill', fontsize=18)

    #adding the text labels
    rects = ax.patches
    #print(rects)
    labels = df.iloc[:,3:].sum().values
    #print(labels)
    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom', fontsize=18)

    plt.show()

In [ ]:
#plot_category_count(df, categories)

### 1.3. Calculating number of jokes having multiple labels

In [54]:
def plot_multiple_label(mlc_labels, multiLabel_counts):
    sns.set(font_scale = 2)
    plt.figure(figsize=(15,8))

    ax = sns.barplot(mlc_labels, multiLabel_counts.values)

    plt.title("Jokes having multiple labels ")
    plt.ylabel('Number of jokes', fontsize=18)
    plt.xlabel('Number of labels', fontsize=18)

    #adding the text labels
    rects = ax.patches
    labels = multiLabel_counts.values
    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

    plt.show()

In [55]:
def print_multiple_label(df):
    rowSums = df.iloc[:,3:].sum(axis=1)
    multiLabel_counts = rowSums.value_counts()
    print(multiLabel_counts)
    multiLabel_counts = multiLabel_counts.iloc[:]
    #print(multiLabel_counts.index)
    mlc_labels = ['L'+str(i) for i in multiLabel_counts.index]
    print(mlc_labels)
    
    #plot_multiple_label(mlc_labels, multiLabel_counts)
    ##return(mlc_labels, multiLabel_counts)

In [56]:
print_multiple_label(df)
print_multiple_label(train_org)
print_multiple_label(train)

1    2964
2     355
3      41
4       3
0       2
dtype: int64
['L1', 'L2', 'L3', 'L4', 'L0']
1    1376
2     270
3      40
4       3
0       2
dtype: int64
['L1', 'L2', 'L3', 'L4', 'L0']
1    1376
2     270
3      40
4       3
0       2
dtype: int64
['L1', 'L2', 'L3', 'L4', 'L0']


## Write the result to a file

In [57]:
with open(outp_path, 'w') as outF:
    outF.write(train.to_csv(sep='\t', index=False))